# LELA70331 Coursework Assignment

This document contains instructions, guidance and code for the coursework assignment for this module.

The assignment focuses on the task of intent classification. You will hear about this in the lecture on dialogue systems. It is an important step in modern task-based dialogue systems - given a particular piece of input from the speaker, the system tries to determine what goal the speaker is trying to achieve, in order that it can then produce an appropriate response.

Your task is to build a system that takes a transcribed user utterance as input and outputs one of seven different intents:

'PlayMusic', e.g. "play easy listening" <br>
'AddToPlaylist' e.g. "please add this song to road trip" <br>
'RateBook' e.g. "give this novel 5 stars"  <br>
'SearchScreeningEvent' e.g. "give me a list of local movie times"  <br>
'BookRestaurant' e.g. "i'd like a table for four at 7pm at Asti"   <br>
'GetWeather' e.g. "what's it like outside"  <br>
'SearchCreativeWork' "show me the new James Bond trailer"  <br>

You are going to evaluate the performance of this system using a test set of 700 user utterances.

In order to create the system you have a training set of 700 utterances and a validation/development set of 700 utterances.

This notebook contains code that you can use in the development of your system. Once you have created and evaluated your system, you are going to write a report of no more than 1500 words that describes and evaluates the task, the system and the experiments.

A guide as to what should go in your report can be found [here]( https://www.dropbox.com/s/7u1yazkok6hmeth/Writing%20your%20Computational%20Linguistics%20Research%20Report%20-%20postgrad.docx?dl=0)

Your report should be submitted via turnitin using the link on Blackboard. You should create a PDF of your notebook and add it as an appendix to your report (not included in word count).  This is just so that I can check what you have done if it isn't clear from your report. You will not be marked on the quality of any code you might include.

Please feel free to ask any questions about any part of the coursework. So that my response can benefit everyone please do so using the Coursework Discussion board on Blackboard. You can find a link to this down the left of the module Blackboard page.

## Preparation

### Link drive

Before you begin to build a system, there are a few steps necessary to set things up. The first of these is to link Colab to your Google Drive so that you can save files there.

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
!mkdir /content/gdrive/My\ Drive/Intent_Classification/

Mounted at /content/gdrive
mkdir: cannot create directory ‘/content/gdrive/My Drive/Intent_Classification/’: File exists


### Download data and some utilities

The next step is to download the data and some supporting code for the project and move it over to the Google Drive.

In [2]:
!wget https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/intent_classification_with_splits.csv
!wget https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/model.pth
!wget https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/nn_tools.py
!wget https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/nn_tools2.py
!wget https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/vectorizer.json

--2023-10-25 09:50:56--  https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/intent_classification_with_splits.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139084 (136K) [text/plain]
Saving to: ‘intent_classification_with_splits.csv’

intent_classificati 100%[===================>] 135.82K  --.-KB/s    in 0.03s   

2023-10-25 09:50:57 (4.38 MB/s) - ‘intent_classification_with_splits.csv’ saved [139084/139084]

--2023-10-25 09:50:57--  https://raw.githubusercontent.com/cbannard/compling23/main/Intent_Classification/model.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

### Import packages

Finally we need to import some packages to use later

In [3]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from nn_tools import Vocabulary, IntentVectorizer, IntentDataset, IntentClassifier
from nn_tools2 import *
from sklearn.metrics import confusion_matrix



## Rule-based approach

Your task here is to create a classifier using rules, in the form of regular expressions. I have provided you with the basic code for doing this. You will just need to edit the regular expressions in order to improve performance.

### Loading and inspecting project data
A valuable first step in order to understand the task is to inspect the data in order to understand the different intents being detected.

First you need to load the data:


In [4]:
intent_data=pd.read_csv('intent_classification_with_splits.csv')

You can then examine example utterances for each of the intent types as follows.

##### Play Music examples


In [5]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "PlayMusic"]["text"].head(10).tolist

<ipython-input-5-99a90d4a3585>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "PlayMusic"]["text"].head(10).tolist


<bound method IndexOpsMixin.tolist of 0     listen to westbam alumb allergic on google music
3                 play the song little robin redbreast
6                  i want to listen to seventies music
7                play a popular chant by brian epstein
13        can you play me some eighties music by adele
30        play the top-20 best chicane songs on deezer
37                play playlist the realest down south
43                             play a chant by mj cole
51               play all of your toys by chris ledoux
52               i want to hear a joel hastings melody
Name: text, dtype: object>

##### AddToPlaylist examples


In [6]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "AddToPlaylist"]["text"].head(10).tolist()

<ipython-input-6-ac5fe56b1c24>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "AddToPlaylist"]["text"].head(10).tolist()


['add step to me to the 50 clásicos playlist',
 'please add iris dement to my playlist this is selena',
 'add slimm cutta calhoun to my this is prince playlist',
 'add to playlist confidence boost here comes santa claus',
 'add another artist to the spotlight on country 2016 playlist',
 'add sugarolly days to my list  your favorite slaughterhouse',
 'add this track to my global funk',
 'add manuelita to my indiespensables playlist',
 'add this artist to gretchen s soul revived playlist',
 'put this song on my playlist  in the name of blues']

##### RateBook examples

In [7]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "RateBook"]["text"].head(10).tolist()

<ipython-input-7-789f5f704f38>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "RateBook"]["text"].head(10).tolist()


['i give this current textbook a rating value of 1 and a best rating of 6',
 'rate this series a 5',
 'give this novel 5 stars',
 'i want to give this current textbook 4 points',
 'rate this album a 1',
 'rate in stars as a 6 for lord of the shadows which gets a four',
 'rate my current book 1 out of 6',
 'for the textbook  out of 6 possible i give the following one a 3',
 'rate this book 0 out of 6',
 'rate the current novel a 3']

##### SearchScreeningEvent examples

In [8]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "SearchScreeningEvent"]["text"].head(10).tolist()

<ipython-input-8-b8e94c0178e1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "SearchScreeningEvent"]["text"].head(10).tolist()


['find fish story',
 'give me a list of movie times for films in the area',
 'find a movie house showing cage without a key',
 'find animated movies nearest at a movie house',
 'find me showtimes for animated movies in the neighbourhood',
 'find movie times',
 'show movie schedules for douglas theatre company',
 'let me see the movie schedule for seed of chucky',
 'in the area find some films',
 'what times will the young swordsman be showing at a local cinema']

##### BookRestaurant examples

In [9]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "BookRestaurant"]["text"].head(10).tolist()

<ipython-input-9-e67137ffa432>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "BookRestaurant"]["text"].head(10).tolist()


['book a spot for 3 in mt',
 'book a restaurant for eight people in six years',
 'i need to book a restaurant in fork mountain  sc for valarie  mari and i',
 'book a restaurant at sixteen o clock in sc',
 'i d like to go to the popular bistro in oh',
 'book a taverna that serves vichyssoise within walking distance in oh',
 'book a brasserie for one',
 'i want to book a restaurant not far from our college',
 'i d like to go to the venetian theatre in gabon  party of seven',
 'book a reservation for a pub with ma po tofu in moldova']

##### GetWeather examples

In [10]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "GetWeather"]["text"].head(10).tolist()

<ipython-input-10-81c0d18a3f60>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "GetWeather"]["text"].head(10).tolist()


['i need a forecast for jetmore  massachusetts in 1 hour and 1 second from now',
 'please let me know the weather forcast of stanislaus national forest far in nine months',
 'what s the weather in my current spot the day after tomorrow',
 'what is the weather like in the city of frewen in the country of venezuela',
 'will it be colder in ohio',
 'what is the weather not far from upper klamath national wildlife refuge',
 'will it storm in charles pinckney national historic site',
 'is it supposed to rain nearby my current location at 0 o clock',
 'what is the weather supposed to be like in new jersey three months from now',
 'will it be warmer now in covenant life']

##### SearchCreativeWork examples

In [11]:
pd.set_option('display.max_colwidth', None)
intent_data[intent_data["split"] == "train"][intent_data["intent"] == "SearchCreativeWork"]["text"].head(10).tolist()

<ipython-input-11-60d6f95a99bb>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  intent_data[intent_data["split"] == "train"][intent_data["intent"] == "SearchCreativeWork"]["text"].head(10).tolist()


['show me the picture creatures of light and darkness',
 'search for the adventures of cookie & cream',
 'play hell house song',
 'i d like to see the show onion sportsdome',
 'find a video game called victory march',
 'find a novel called industry',
 'please find the movie dancing girl',
 'find resurrection of evil',
 'find playstation官方杂志  a song',
 'find young miss holmes']

## Building a rule based classifier

### Preprocessing

To increase the generalisability of your system you can preprocess it to, for example, convert morphological variants to a single "lemma". You can do this by adding different substitution (re.sub) functions to the function below. The function as is stands doesn't do anything - you need to update the re.sub statements. This isn't an essential step as you can deal with variants in your patterns, but it will help to reduce redundancy in your classifier regular expressions which you may find helpful.


In [12]:
def preprocess_utterance(utt):
  utt = re.sub("","", utt)
  utt = re.sub("","", utt)
  return utt

You can test whether you patterns are working as intented using the following code.

In [13]:
test_input = input("Enter a utterance to test your preprocessing on: ")
print(preprocess_utterance(test_input))

Enter a utterance to test your preprocessing on: play jazz
play jazz


### Define patterns

The function below takes an utterance as input and applies a series of regular expressions to identify the intent of the speaker. The regular expressions currently just looks for keywords taken from the intent name. You should update these patterns to be more appropriate and capture a wider range of utterances for each intent.

Each time you update the code you will need to run the code cell in order to then use the function.

The assign_intent function uses the re.findall function (see week 2) in order to make as many matches as possible with each pattern. The number of matches is then counted (using the len function) for each pattern. The intent with the largest number of matches is then returned as the predicted intent. Imagine for example that your patterns for PlayMusic and GetWeather were as follows: <br>
PlayMusic_Pattern = re.compile("play|music") <br>
GetWeather_Pattern = re.compile("get|weather") <br>
while the input utterances was "play the weather girls".
In this case the PlayMusic pattern would match twice (for play and music) while the GetWeather pattern would only match once. PlayMusic would be returned as the predicted intent. Where there is a tie a prediction is randomly sampled from among the tied intents.

In [14]:
def assign_intent(utt, verbose=False):
  PlayMusic_Pattern = re.compile("play|music")
  AddToPlaylist_Pattern = re.compile("add|playlist")
  RateBook_Pattern = re.compile("rate|book")
  SearchScreeningEvent_Pattern = re.compile("screening")
  BookRestaurant_Pattern = re.compile("book|restaurant")
  GetWeather_Pattern = re.compile("get|weather")
  SearchCreativeWork_Pattern = re.compile("creative")

  weights = {}
  weights['PlayMusic'] = len(re.findall(PlayMusic_Pattern,  utt))
  weights['AddToPlaylist'] = len(re.findall(AddToPlaylist_Pattern,  utt))
  weights['RateBook'] = len(re.findall(RateBook_Pattern,  utt))
  weights['SearchScreeningEvent'] = len(re.findall(SearchScreeningEvent_Pattern,  utt))
  weights['BookRestaurant'] = len(re.findall(BookRestaurant_Pattern,  utt))
  weights['GetWeather'] = len(re.findall(GetWeather_Pattern,  utt))
  weights['SearchCreativeWork'] = len(re.findall(SearchCreativeWork_Pattern,  utt))
  if verbose:
      print(weights)
  if max(weights.values()) == 0:
      return random.choice(list(weights.keys()))
  else:
      weights_as_list = list(weights.items())
      random.shuffle(weights_as_list)
      weights=dict(weights_as_list)
      return max(weights, key=lambda key: weights[key])

### Evaluation
When you run this cell you will be asked to enter an utterance. When you press return the scores for each classification of your input will be printed. You can use this to check whether your preprocess_utterance and/or assign_intent functions are working as intended.

In [15]:
new_input = input("Enter a utterance to classify: ")
prediction = assign_intent(preprocess_utterance(new_input),verbose=True)
print(prediction)

Enter a utterance to classify: play jazz
{'PlayMusic': 1, 'AddToPlaylist': 0, 'RateBook': 0, 'SearchScreeningEvent': 0, 'BookRestaurant': 0, 'GetWeather': 0, 'SearchCreativeWork': 0}
PlayMusic


In order to perform a stricter assessment of the performance of your classifier, you should examine its performance on the validation dataset. If you run the cell below you will be told the accuracy score on those 700 utterances.

In [16]:
predicted = [assign_intent(preprocess_utterance(item)) for item in intent_data[intent_data['split'] == "val"]['text']]
true = intent_data[intent_data['split'] == "val"]['intent']
accuracy = (predicted == true).sum()/len(true)
print(accuracy)

0.5371428571428571


Running the next cell will give you a "confusion matrix". This tells you the number of times that each intent as given in the rows is classified (correctly or otherwise) as each intent as represented by the columns. The columns are displayed as numbers but you can check which each of these numbers stands for by looking into the parentheses after each intent in the rows.

Studying this will give you an idea of where the classifier might be going wrong, and therefore of how you might update your patterns.

In [17]:
print(pd.DataFrame(confusion_matrix(predicted, true), index=["AddToPlaylist (0)", "BookRestaurant (1)", "GetWeather (2)", "PlayMusic (3)", "RateBook (4)", "SearchCreativeWork (5)", "SearchScreeningEvent (6)"]))

                           0   1   2   3   4   5   6
AddToPlaylist (0)         92   3   5   0   7  16   8
BookRestaurant (1)         0  53  11   5   9   4  10
GetWeather (2)             1   1  52   2   7  22  16
PlayMusic (3)              5   2   9  90   4  13  34
RateBook (4)               0  38   8   1  63  17  14
SearchCreativeWork (5)     0   1   9   1   5  17   9
SearchScreeningEvent (6)   2   2   6   1   5  11   9


Once you are happy that you have defined the best patterns that you can, you should evaluate the performance of your classifier on the test data (a set of 700 utterances that you haven't looked at). The accuracy printed is what you should include in your report.

In [18]:
predicted = [assign_intent(preprocess_utterance(item)) for item in intent_data[intent_data['split'] == "test"]['text']]
true = intent_data[intent_data['split'] == "test"]['intent']
accuracy = (predicted == true).sum()/len(true)
print(accuracy)

0.5357142857142857


You can also generate a confusion matrix for the test data and use this in the discussion of the results in your write up.

In [19]:
print(pd.DataFrame(confusion_matrix(predicted, true), index=["AddToPlaylist (0)", "BookRestaurant (1)", "GetWeather (2)", "PlayMusic (3)", "RateBook (4)", "SearchCreativeWork (5)", "SearchScreeningEvent (6)"]))

                            0   1   2   3   4   5   6
AddToPlaylist (0)         110   6  13   2   1  16   6
BookRestaurant (1)          1  47   5   0   6   6  14
GetWeather (2)              2   4  50   4   3  14  16
PlayMusic (3)              10   2  14  77   6  24  41
RateBook (4)                0  27  10   0  58  13   7
SearchCreativeWork (5)      1   2   6   2   4  18   8
SearchScreeningEvent (6)    0   4   6   1   2  16  15


## Single Layer Perceptron Classifier

The next classifier you can build is single-layer perceptron. The specification of this model is in the next cell. You shouldn't make any changes to this.

In [20]:
class IntentClassifierPerceptron(nn.Module):
    """ a simple perceptron based classifier """
    def __init__(self, input_dim, output_dim):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(IntentClassifierPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)




    def forward(self, x_in, apply_softmax=True):
        """The forward pass of the classifier

        Args:
            x_in (torch.Tensor): an input data tensor.
                x_in.shape should be (batch, num_features)
            apply_softmax (bool): a flag for the softmax activation
        Returns:
            the resulting tensor. tensor.shape should be (batch,)
        """

        y_out = self.fc1(x_in)
        if apply_softmax:
            y_out = F.softmax(y_out,dim=1)
        return y_out


### Preprocessing
As with the rule-based model, to increase the generalisability of your system you can preprocess the text to, for example, convert morphological variants to a single "lemma". You can do ths by adding different substitution (re.sub) functions to the function below. The function as is stands doesn't do anything - you need to update the re.sub statements.


In [21]:
def preprocess_utterance(utt):
  utt = re.sub("","", utt)
  return utt

For this machine-learning-based model we are going to make the preprocessing changes to the data used, and this will involve making changes to the file we have saved.

Before we make changes to the data however, we should create a backup of the unaltered file in case we need to revert to it.

In [22]:
!cp intent_classification_with_splits.csv /content/gdrive/My\ Drive/Intent_Classification/

When we need to revert to the original unaltered file, we can then run the following cell.

In [23]:
!cp /content/gdrive/My\ Drive/Intent_Classification/intent_classification_with_splits.csv .
intent_data=pd.read_csv('/content/gdrive/My Drive/Intent_Classification/intent_classification_with_splits.csv')

You can check that your preprocessing patterns are doing what we want by testing them on examples using the following cell

In [24]:
test_input = input("Enter a utterance to test your preprocessing on: ")
print(preprocess_utterance(test_input))

Enter a utterance to test your preprocessing on: play jazz
play jazz


Once you are happy with your preprocessing you can then transform the text in the training, validation and test data using the following cell. This alters the data on the disk so if you want to undo any changes later you will have to revert to the original data (as described above).

In [25]:
intent_data['text'] = [preprocess_utterance(item) for item in intent_data['text']]
intent_data.to_csv('intent_classification_with_splits.csv', index=False)

### Training the model
In order to first initialise your model and then train it, you should run the following cell. The training will take a minute or two to complete - the progress bars will tell you how far along it is.


In [26]:
params = initialise()
classifier = IntentClassifierPerceptron(input_dim=len(params.vectorizer.text_vocab),output_dim=len(params.vectorizer.intent_vocab))
train_state = trainModel(params, params.dataset, classifier)

Expanded filepaths: 
	/content/gdrive/My Drive/Intent_Classification/vectorizer.json
	/content/gdrive/My Drive/Intent_Classification/model.pth
Using CUDA: False


training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/5 [00:00<?, ?it/s]

split=val:   0%|          | 0/5 [00:00<?, ?it/s]

### Test on individual utterance

As with the rule-based classifier you can look at the performance of the system on a single example utterance. This can be used to see whether your preprocessing is helping to capture the cases you hoped.

In [27]:
torch.manual_seed(0)
new_utterance = input("Enter a utterance to classify: ")
classifier = classifier.to("cpu")
prediction = predict_intent(preprocess_utterance(new_utterance), classifier, params.vectorizer)
print("{} -> {} (p={:0.2f})".format(new_utterance,
                                    prediction['intent'],
                                    prediction['probability']))

Enter a utterance to classify: play jazz
play jazz -> PlayMusic (p=0.37)


### Evaluate performance on validation data
In order to evaluate the performance of your system while you tweak your preprocessing you can evaluate performance on the validation data, and look at a confusion matrix.

In [28]:
torch.manual_seed(0)
predicted, true = evaluate(params, classifier, train_state, 'val')
print("Test Accuracy: {:.2f}".format(train_state['val_acc']))
print(pd.DataFrame(confusion_matrix(predicted, true), index=["AddToPlaylist", "BookRestaurant", "GetWeather", "PlayMusic", "RateBook", "SearchCreativeWork", "SearchScreeningEvent"]))

Test Accuracy: 82.34
                       0   1   2   3   4   5   6
AddToPlaylist         84   0   0   2   0   5   0
BookRestaurant         0  96   2   2   1   8  14
GetWeather             1   0  94   1   0   8  25
PlayMusic              1   1   1  79   1   7   4
RateBook               1   0   0   0  93   8   1
SearchCreativeWork     0   0   0   0   0  45   6
SearchScreeningEvent   0   0   0   0   0  13  36


### Evaluate performance on test data

Once you are happy that model performance is as good as you can achieve with this model type you should evaluate its accuracy on the test data. You can also use the confusion matrix for error analysis in your write up.

In [29]:
torch.manual_seed(0)
predicted, true = evaluate(params, classifier, train_state, 'test')
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))
print(pd.DataFrame(confusion_matrix(predicted, true), index=["AddToPlaylist", "BookRestaurant", "GetWeather", "PlayMusic", "RateBook", "SearchCreativeWork", "SearchScreeningEvent"]))

Test Accuracy: 80.00
                        0   1   2   3   4   5   6
AddToPlaylist         104   1   1   4   0   3   1
BookRestaurant          2  85   5   0   0   9  10
GetWeather              1   0  87   1   1   3  25
PlayMusic               3   1   0  73   1  13   2
RateBook                1   0   0   1  68  13   5
SearchCreativeWork      0   0   1   0   0  47   8
SearchScreeningEvent    0   0   0   0   0  12  48


## Multilayer neural network classifier

The third kind of classifier that you should build is a two-layer neural network. The model is defined below. Again you don't need to change this code.

In [30]:
class IntentClassifierMLP(nn.Module):
    """ a simple perceptron based classifier """
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(IntentClassifierMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)



    def forward(self, x_in, apply_softmax=True):
        """The forward pass of the classifier

        Args:
            x_in (torch.Tensor): an input data tensor.
                x_in.shape should be (batch, num_features)
            apply_softmax (bool): a flag for the softmax activation
        Returns:
            the resulting tensor. tensor.shape should be (batch,)
        """
        intermediate_vector = F.relu(self.fc1(x_in))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector,dim=1)
        return prediction_vector


### Preprocessing
As with the previous models, to increase the generalisability of your system you can preprocess the text to, for example, convert morphological variants to a single "lemma". You can do ths by adding different substitution (re.sub) functions to the function below. The function as is stands doesn't do anything - you need to update the re.sub statements.


In [31]:
def preprocess_utterance(utt):
  utt = re.sub("","", utt)
  utt = re.sub("","", utt)
  utt = re.sub("","", utt)
  return utt

For this machine-learning-based model we are going to make the preprocessing changes to the data use, and this will involve making changes to the file we have saved.

Before we make changes to the data however, we should create a backup of the unaltered file in case we need to revert to it.

In [32]:
!cp intent_classification_with_splits.csv /content/gdrive/My\ Drive/Intent_Classification/

When we need to revert to the original unaltered file, we can then run the following cell.

In [33]:
!cp /content/gdrive/My\ Drive/Intent_Classification/intent_classification_with_splits.csv .
intent_data=pd.read_csv('intent_classification_with_splits.csv')

You can check that your preprocessing patterns are doing what we want by testing them on examples using the following cell

In [34]:
test_input = input("Enter a utterance to test your preprocessing on: ")
print(preprocess_utterance(test_input))


Enter a utterance to test your preprocessing on: play jazz
play jazz


Once you are happy with your preprocessing you can then transform the text in the training, validation and test data using the following cell. This alters the data on the disk so if you want to undo any changes later you will have to revert to the original data (as described above).

In [35]:
intent_data['text'] = [preprocess_utterance(item) for item in intent_data['text']]
intent_data.to_csv('intent_classification_with_splits.csv', index=False)

### Training model
You can now move on to training the model using your preprocessed data.

One important variable that you can change is the number of nodes to include in your hidden layer. You can set this parameter in the cell below. The default is 100.

In [36]:
n_hidden_dims = 100

You can then initialise and train the model by running the following cell.

In [37]:
params = initialise()
classifier = IntentClassifierMLP(input_dim=len(params.vectorizer.text_vocab),hidden_dim=n_hidden_dims,output_dim=len(params.vectorizer.intent_vocab))
train_state = trainModel(params, params.dataset, classifier)

Expanded filepaths: 
	/content/gdrive/My Drive/Intent_Classification/vectorizer.json
	/content/gdrive/My Drive/Intent_Classification/model.pth
Using CUDA: False


training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/5 [00:00<?, ?it/s]

split=val:   0%|          | 0/5 [00:00<?, ?it/s]

The following cell allows you to look at the performance of the system on a single example utterance. This can be used to see whether your preprocessing (and/or your choice of hidden dimensions) is capturing the cases you hoped.

In [38]:
torch.manual_seed(0)
new_utterance = input("Enter a utterance to classify: ")
classifier = classifier.to("cpu")
prediction = predict_intent(preprocess_utterance(new_utterance), classifier, params.vectorizer)
print("{} -> {} (p={:0.2f})".format(new_utterance,
                                    prediction['intent'],
                                    prediction['probability']))

Enter a utterance to classify: play country
play country -> PlayMusic (p=0.96)


### Evaluate model on validation data

In order to evaluate the performance of your system while you tweak it you can evaluate performance on the validation data, and look at a confusion matrix.

In [39]:
torch.manual_seed(0)
predicted, true = evaluate(params, classifier, train_state, 'val')
print("Test Accuracy: {:.2f}".format(train_state['val_acc']))
print(pd.DataFrame(confusion_matrix(predicted, true), index=["AddToPlaylist", "BookRestaurant", "GetWeather", "PlayMusic", "RateBook", "SearchCreativeWork", "SearchScreeningEvent"]))

Test Accuracy: 89.06
                       0   1   2   3   4   5   6
AddToPlaylist         84   0   0   0   0   0   0
BookRestaurant         0  95   2   2   1   2   9
GetWeather             1   1  79   0   0   3   5
PlayMusic              1   0   1  80   0   2   0
RateBook               0   0   0   0  94   0   0
SearchCreativeWork     1   0   1   1   0  81  15
SearchScreeningEvent   0   1  14   1   0   6  57


### Evaluate model on test data

Once you are happy that model performance is as good as you can achieve with this model type you should evaluate its accuracy on the test data. You can also use the confusion matrix for error analysis in your write up.

In [40]:
torch.manual_seed(0)
predicted, true = evaluate(params, classifier, train_state, 'test')
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))
print(pd.DataFrame(confusion_matrix(predicted, true), index=["AddToPlaylist", "BookRestaurant", "GetWeather", "PlayMusic", "RateBook", "SearchCreativeWork", "SearchScreeningEvent"]))

Test Accuracy: 87.19
                        0   1   2   3   4   5   6
AddToPlaylist         103   0   0   0   0   0   1
BookRestaurant          1  79   3   0   1   2   1
GetWeather              1   3  81   0   0   0   0
PlayMusic               3   1   0  75   2  12   0
RateBook                0   0   0   0  67   1   1
SearchCreativeWork      3   2   1   3   0  75  18
SearchScreeningEvent    0   2   9   1   0  10  78


## That's it!

Once you have run through all of this notebook, made all of the additions and changes you want, and run all of the experiments you want you should write up the results following the guidance [here]( https://www.dropbox.com/s/7u1yazkok6hmeth/Writing%20your%20Computational%20Linguistics%20Research%20Report%20-%20postgrad.docx?dl=0)

Your report should be submitted via turnitin using the link on Blackboard.

To repeat a couple of things from the top of the sheet:

- Once you have completed your work in this notebook, please use print to create a PDF file and attach it to your report when you submit it. This is just so that I can check what you have done if it isn't clear from your report. You will not be marked on the quality of any code you might include.

- Please feel free to ask any questions about any part of the coursework. So that my response can benefit everyone please do so using the Coursework Discussion board on Blackboard. You can find a link to this down the left of the module Blackboard page.

I hope you find it an enjoyable and worthwhile exercise.
